In [2]:
import polars as pl
from html.parser import HTMLParser

In [16]:
df = pl.read_parquet("C:\labs\hh_hack\hh_recsys_train_hh.pq")
labels = pl.read_parquet("C:\labs\hh_hack\hh_vac2idx_data.pq")
vac = pl.read_parquet("C:\labs\hh_hack\hh_recsys_vacancies.pq")

In [23]:
labels.head()

vacancy_id,idx
str,i64
"""v_1001643""",0
"""v_1002507""",1
"""v_1004084""",2
"""v_1005147""",3
"""v_1005189""",4


In [21]:
vac2idx = {labels["vacancy_id"][i]: int(labels["idx"][i]) for i in range(labels.shape[0])}

In [28]:
[i for i in vac2idx]

['v_1001643',
 'v_1002507',
 'v_1004084',
 'v_1005147',
 'v_1005189',
 'v_1005231',
 'v_100545',
 'v_1006183',
 'v_1006816',
 'v_1007526',
 'v_100795',
 'v_1008427',
 'v_1008928',
 'v_1010890',
 'v_1010894',
 'v_101119',
 'v_1011703',
 'v_1011942',
 'v_1011956',
 'v_101243',
 'v_1013173',
 'v_1015631',
 'v_1016011',
 'v_1016907',
 'v_1017022',
 'v_1018123',
 'v_1018159',
 'v_1019413',
 'v_101985',
 'v_102114',
 'v_1021332',
 'v_1021638',
 'v_1021825',
 'v_1022229',
 'v_1022594',
 'v_1025606',
 'v_1026630',
 'v_1027775',
 'v_1028508',
 'v_1028853',
 'v_10289',
 'v_1030003',
 'v_1030014',
 'v_1032043',
 'v_1032093',
 'v_1032816',
 'v_1033430',
 'v_1034177',
 'v_1034310',
 'v_1034517',
 'v_1035409',
 'v_1035492',
 'v_1038828',
 'v_1042065',
 'v_1043562',
 'v_1043717',
 'v_1044555',
 'v_1045102',
 'v_1045667',
 'v_1046246',
 'v_1047886',
 'v_1048191',
 'v_1048719',
 'v_1048781',
 'v_1048838',
 'v_1049158',
 'v_104939',
 'v_1049848',
 'v_1049953',
 'v_1051057',
 'v_1051612',
 'v_1051777',
 

In [27]:
new_vac = vac.filter(pl.col("vacancy_id").is_in(labels["vacancy_id"]))
new_vac = new_vac.with_columns(new_vac["vacancy_id"].replace(vac2idx).cast(int))
new_vac.filter(pl.col("vacancy_id")==0)

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
i64,str,str,str,list[str],i64,i64,str,str,str,str,str,str
0,"""Менеджер по пр…","""c_5770""","""<p>Компания ПУ…",null,null,null,null,"""a_5657""","""ar_8""","""full""","""fullDay""","""between1And3"""


In [31]:
t = pl.read_parquet("C:\labs\hh_hack\extracted_features.pq")

In [32]:
t

vacancy_id,name,description,keySkills.keySkill,compensation.from,compensation.to,area.id,employment,workSchedule,workExperience
str,str,list[str],list[str],i64,i64,i64,i64,i64,i64
"""v_862116""","""Смотритель муз…","[""Обязанности:"", ""Осуществлять"", … ""скользящий""]","[""Пользователь ПК"", ""Работа в команде"", … ""PR-консультирование""]",16500,null,4181,0,2,3
"""v_288642""","""Ведущий менедж…","[""Возможно,"", ""наша"", … ""зарабатывать.""]","[""Активные продажи"", ""Холодные продажи"", … ""Организация мероприятий""]",50000,null,6731,0,2,3
"""v_1840054""","""Бухгалтер (по …","[""Обязанности:"", ""Расчет"", … ""труда.""]","[""""]",50000,65000,5806,0,2,1
"""v_2346232""","""Пекарь (Токсов…","[""Для"", ""каждого,"", … ""блинопек.""]","[""""]",38500,42000,4218,0,2,3
"""v_312507""","""Торговый предс…","[""Компания"", ""ТД"", … ""единомышленников!""]","[""Продуктивность"", ""Клиентоориентированность"", … ""Развитие продаж""]",60000,null,6487,0,2,0
"""v_2323706""","""Менеджер по ка…","[""X5"", ""READY"", … ""27/1.""]","[""Контроль качества"", ""Производственный контроль"", … ""Лабораторная диагностика""]",88000,null,5469,0,2,0
"""v_704266""","""Менеджер по пр…","[""Если"", ""ты"", … ""Life.""]","[""Активные продажи"", ""Поиск и привлечение клиентов"", … ""Проведение переговоров""]",30000,100000,4888,0,2,0
"""v_525733""","""Врач по ЛФК""","[""Обязанности:"", ""Высшее"", … ""неделя.""]","[""""]",0,null,4495,0,2,1
"""v_2091086""","""Монтажник СТиЖ…","[""Строительная"", ""компания"", … ""БЕТОНЩИКИ.""]","[""""]",100000,null,4447,0,1,0


In [7]:
# from prompt_toolkit import print_formatted_text, HTML

# print_formatted_text(HTML("'<strong>Обязанности:</strong> <ul> <li>Осуществлять контроль за соблюдением посетителями правил поведения в музее</li> <li>При возникновении угрозы повреждения или кражи музейных предметов информировать руководство музея</li> </ul> <strong>Требования:</strong> <ul> <li>образование средне-специальное или высшее</li> <li>музейное дело, культурология</li> </ul> <strong>Условия:</strong> <ul> <li>Выходные: 2 дня. График с 09:00 до 18:00, по четвергам с 11:00 до 20:00</li> <li>Обед скользящий</li> </ul>'"))

cur_code2rus = {"KZT": "₸", "BYR": "Br.", "EUR": "€", "KGS": "som", "RUR": "₽", "USD": "$", "UZS": "so'm"}

sch2rus = {
    "fullDay": "Полный рабочий день",
    "remote": "Удаленная работа",
    "flexible": "Гибкий график",
    "shift": "Работа по сменам",
    "flyInFlyOut": "Работа на вылет"
}

empl2rus = {
    "part": "Частичная занятость",
    "probation": "Испытательный срок",
    "full": "Полная занятость",
    "project": "Проектная работа",
    "volunteer": "Волонтерство"
}

exp2rus = {
    "moreThan6": "более 6 лет",
    "between1And3": "от 1 до 3 лет",
    "between3And6": "от 3 до 6 лет",
    "noExperience": "без опыта работы"
}

t = 0

def VacancyInfo(i = t) -> str:
    parse_vac = df[i]
    name = parse_vac["name"].item()
    description = parse_vac["description"].item()
    compensation_from = parse_vac["compensation.from"].item() if not (
                parse_vac["compensation.from"].item() is None) else 0
    compensation_to = parse_vac["compensation.to"].item() if not (
                parse_vac["compensation.to"].item() is None) else compensation_from
    res_compensation = f"{compensation_from} - {compensation_to}" if compensation_from != compensation_to else compensation_from
    compensation_code = cur_code2rus[parse_vac["compensation.currencyCode"].item()] if not (
                parse_vac["compensation.currencyCode"].item() is None) else ""
    skills = ", ".join(parse_vac["keySkills.keySkill"].item().to_list()) if not (
                parse_vac["keySkills.keySkill"].item() is None) else "Ключевые навыки не требуются"
    employment = empl2rus[parse_vac["employment"].item()]
    parse_work_sch = sch2rus[parse_vac["workSchedule"].item()]
    exp = exp2rus[parse_vac["workExperience"].item()]

    vacancy_info = f"<b>{name}</b>\n\n"
    vacancy_info += f"<b>Описание:</b>\n{description}\n\n"
    vacancy_info += f"<b>Условия:</b>\n{parse_work_sch}\n"
    vacancy_info += f"<b>Опыт работы:</b> {exp}\n"
    vacancy_info += f"<b>Трудоустройство:</b> {employment}\n"
    vacancy_info += f"<b>Зарплата:</b> {res_compensation} {compensation_code}\n"
    vacancy_info += f"<b>Ключевые навыки:</b> {skills}\n"

    return vacancy_info

print()

TypeError: 'module' object is not callable

In [ ]:
"'<strong>Обязанности:</strong> <ul> <li>Осуществлять контроль за соблюдением посетителями правил поведения в музее</li> <li>При возникновении угрозы повреждения или кражи музейных предметов информировать руководство музея</li> </ul> <strong>Требования:</strong> <ul> <li>образование средне-специальное или высшее</li> <li>музейное дело, культурология</li> </ul> <strong>Условия:</strong> <ul> <li>Выходные: 2 дня. График с 09:00 до 18:00, по четвергам с 11:00 до 20:00</li> <li>Обед скользящий</li> </ul>'"

In [74]:
s = ""
class MyHTMLParser(HTMLParser):
    def handle_data(self, data):
        # print(data)
        global s
        s+=data+"\n\n" if data != " " else ""
    

In [76]:
p = MyHTMLParser()
p.feed("<strong>Обязанности:</strong> <ul> <li>Осуществлять контроль за соблюдением посетителями правил поведения в музее</li> <li>При возникновении угрозы повреждения или кражи музейных предметов информировать руководство музея</li> </ul> <strong>Требования:</strong> <ul> <li>образование средне-специальное или высшее</li> <li>музейное дело, культурология</li> </ul> <strong>Условия:</strong> <ul> <li>Выходные: 2 дня. График с 09:00 до 18:00, по четвергам с 11:00 до 20:00</li> <li>Обед скользящий</li> </ul>")

In [77]:
print(s)

Обязанности:

Осуществлять контроль за соблюдением посетителями правил поведения в музее

При возникновении угрозы повреждения или кражи музейных предметов информировать руководство музея

Требования:

образование средне-специальное или высшее

музейное дело, культурология

Условия:

Выходные: 2 дня. График с 09:00 до 18:00, по четвергам с 11:00 до 20:00

Обед скользящий




In [78]:
p.handle_data("<strong>Обязанности:</strong> <ul> <li>Осуществлять контроль за соблюдением посетителями правил поведения в музее</li> <li>При возникновении угрозы повреждения или кражи музейных предметов информировать руководство музея</li> </ul> <strong>Требования:</strong> <ul> <li>образование средне-специальное или высшее</li> <li>музейное дело, культурология</li> </ul> <strong>Условия:</strong> <ul> <li>Выходные: 2 дня. График с 09:00 до 18:00, по четвергам с 11:00 до 20:00</li> <li>Обед скользящий</li> </ul>")

In [79]:
p